In [ ]:
#import requests
#url = 'https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv'
#content = requests.get(url)

# Application d'aide à la consommation

## Source variable

**- Définition de nos variable:**
https://world.openfoodfacts.org/data/data-fields.txt


## Objectif :
- Repondre à un appel à projets pour trouver des idées innovantes d’applications en lien avec l'alimentation:

  **Idée_1:** Application proposant des boissons ayant un meillleur nutriscore/ - additifs lorsque l'on scan un   produit. Si le nutriscore n'est pas présent faire une estimation en fonction des ingrédients + label + additifs?

  **Idée_2:** Application proposant des produits similaires allergènes (sans allergènes) pour les personnes ayant des allergies. Ces personnes rentrerai les types d'allergies directement sur leur profil dans l'app
  
  **Catégorie:** Mot clés des most comon word consernant les boissons
  
  **Pays:** France car l'appel d'offre concerne l'agence "Santé publique France" ??
  
  **Mot clés:** Boisson, Drink, Nutriscore ???

  
- Traiter le jeu de données afin de repérer des variables pertinentes pour les traitements à venir. Automatiser ces traitements pour éviter de répéter ces opérations.
  Le programme doit fonctionner si la base de données est légèrement modifiée (ajout d’entrées, par exemple).
    

- Confirmer ou infirmer les hypothèses  à l’aide d’une analyse multivariée. Effectuer les tests statistiques appropriés pour vérifier la significativité des résultats.

- Élaborer une idée d’application. Identifier des arguments justifiant la faisabilité (ou non) de l’application à partir des données Open Food Facts.


## Checklist de base
#### Analyse de Forme :
- **variable target** : **nutriscore_grade**
- **variable subtarget** : **categories_tags**, **additives_n**
- **variable subtarget** :   
    
    ***nutriscore_grade*** Calculer à partir de (source: https://www.economie.gouv.fr/entreprises/nutri-score):
    - valeur énergétique----> energy-kcal_100g [* 4.184 convert kcal to kj]
    - quantité de graisses--> fat_100g
    - acides gras saturés---> saturated-fat_100g 
    - glucides--------------> carbohydrates_100g
    - sucres----------------> sugars_100g
    - sel-------------------> salt_100g
    - fibre-----------------> fiber_100g
    - proteine--------------> proteins_100g

- **lignes et colonnes** : 1705429, 184
- **types de variables** : qualitatives: 59, quantitatives: 125
- **Analyse des valeurs manquantes** : 

    Beaucoup de colonnes avec + de 90% de variables manquantes
    
    Colonne sans variable: ingredients_from_palm_oil, nutrition-score-uk_100g, water-hardness_100g, no_nutriments, allergens_en, cities, ingredients_that_may_be_from_palm_oil, additives


In [1]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def reduce_mem_usage(df, verbose=True):
    """
    Fonction visant à réduire l'utilisation de la mémoire par le dataframe, en optimisant les dtypes. 
    source: "https://gist.github.com/eherrerosj/fdac2b3761a10332da163b1c90367890"
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def most_common_words(labels):
    words = []
    for lab in labels:
        words += lab.split(",")
    counter = Counter(words)
    for word in counter.most_common(300):
        print(word)

In [16]:
data = pd.read_csv("./../en.openfoodfacts.org.products.csv", low_memory=False, encoding='utf-8', sep='\t')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705429 entries, 0 to 1705428
Columns: 184 entries, code to carnitine_100g
dtypes: float64(123), int64(2), object(59)
memory usage: 2.3+ GB


In [4]:
data.columns.values

array(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'abbreviated_product_name', 'generic_name', 'quantity',
       'packaging', 'packaging_tags', 'packaging_text', 'brands',
       'brands_tags', 'categories', 'categories_tags', 'categories_en',
       'origins', 'origins_tags', 'origins_en', 'manufacturing_places',
       'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en',
       'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo',
       'cities', 'cities_tags', 'purchase_places', 'stores', 'countries',
       'countries_tags', 'countries_en', 'ingredients_text', 'allergens',
       'allergens_en', 'traces', 'traces_tags', 'traces_en',
       'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n',
       'additives', 'additives_tags', 'additives_en',
       'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil',
       'ingredients_from_palm_oil_tags',
       

In [7]:
reduce_mem_usage(data)

Mem. usage decreased to 1307.64 Mb (45.4% reduction)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,NaN,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705424,9999999910128,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1572513866,2019-10-31T09:24:26Z,1572513866,2019-10-31T09:24:26Z,Sandwich club Rillette poisson combava,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705425,999999999,http://world-en.openfoodfacts.org/product/9999...,nutrinet-sante,1608105503,2020-12-16T07:58:23Z,1608105504,2020-12-16T07:58:24Z,Thé noir BIO Darjeeling,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705426,9999999990397,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1581171613,2020-02-08T14:20:13Z,1582644247,2020-02-25T15:24:07Z,Fati,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705427,9999999999994,http://world-en.openfoodfacts.org/product/9999...,kiliweb,1613129728,2021-02-12T11:35:28Z,1613129730,2021-02-12T11:35:30Z,Light & Free SKYR A BOIRE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
with pd.option_context('display.max_rows', len(data.columns)):    
    print((data.isna().sum()/data.shape[0]).sort_values(ascending=True))

code                                          0.000000
url                                           0.000000
created_t                                     0.000000
created_datetime                              0.000000
last_modified_t                               0.000000
last_modified_datetime                        0.000000
states_en                                     0.000000
states_tags                                   0.000000
states                                        0.000000
creator                                       0.000002
pnns_groups_2                                 0.000350
countries                                     0.003185
countries_en                                  0.003187
countries_tags                                0.003187
pnns_groups_1                                 0.009760
product_name                                  0.045228
energy_100g                                   0.205644
proteins_100g                                 0.209859
fat_100g  

In [18]:
del(data)

# Nettoyage